In [4]:
import numpy as np
import os
import pandas as pd

In [5]:
pixel_count = pd.read_csv(os.path.join(os.getcwd(), 'rasters_2020_pixel_counts.csv'))
pixel_count

,n_nonice_20,n_ice_20,n_ground_20,n_water_20,raster
0,32334429,9319051,111150412,62968690,LS_merged_crs26910_S_2020
1,887994,264213,1891593,2893071,LS_merged_crs26910_W_2020
2,87881212,2912345,62587031,69125241,LS_merged_crs26911_2020_clip


In [13]:
# total number of pixels in neach category
n_pix = [sum(pixel_count.iloc[:,c]) for c in range(0,4)]
n_pix

[121103635, 12495609, 175629036, 134987002]

In [14]:
# Recreation of sampling design by SEPAL

# ---------------------------------------------
# --------------- PARAMETERS ------------------
# standard error for all the points
std_error = 0.015

# user's accuracies TP/(TP+FP) (estimates)
U = [0.7, 0.6, 0.9, 0.95]
# ---------------------------------------------

# fraction of pixels with a given class in total pixels
total_pix = sum(n_pix)
pix_prop = [n/total_pix for n in n_pix]

# standard deviation of user's accuracies
stdv = [ np.sqrt(u*(1-u)) for u in U]

X = [ x*y for x,y in zip(pix_prop, stdv)]

sample_size = (sum(X)/std_error)**2
sample_size

# distributing sample size among classes
#[...]

465.27022176992

In [15]:
errors_U =[]
strat_samples = []
strat_title = []

def strat_stderror(U, strat_sample):
    return [ np.sqrt(u*(1-u)/(n-1)) for u,n in zip(U, strat_sample) ]

In [20]:
# Distributing sample size among classes

# ---------------------------------------------
strat_title.append('equal')
sample_equal = [sample_size/4 for i in range(0,4)]
strat_samples.append(sample_equal)
errors_U.append(strat_stderror(U, sample_equal))

# ---------------------------------------------
class2_prop = n_pix[2]/(n_pix[2]+n_pix[3])
class3_prop = n_pix[3]/(n_pix[2]+n_pix[3])

# ---------------------------------------------
strat_title.append('120')
sample_120 = [120, 120, (sample_size-240)*class2_prop, (sample_size-240)*class3_prop] 
strat_samples.append(sample_120)
errors_U.append(strat_stderror(U, sample_120))

# ---------------------------------------------
strat_title.append('100')
sample_100 = [100, 100, (sample_size-200)*class2_prop, (sample_size-200)*class3_prop] 
strat_samples.append(sample_100)
errors_U.append(strat_stderror(U, sample_100))

# ---------------------------------------------
strat_title.append('87')
sample_87 = [87, 87, (sample_size-174)*class2_prop, (sample_size-174)*class3_prop] 
strat_samples.append(sample_87)
errors_U.append(strat_stderror(U, sample_87))

# ---------------------------------------------
strat_title.append('75')
sample_75 = [75, 75, (sample_size-150)*class2_prop, (sample_size-150)*class3_prop] 
strat_samples.append(sample_75)
errors_U.append(strat_stderror(U, sample_75))

# ---------------------------------------------
strat_title.append('prop')
sample_prop = [sample_size*x for x in pix_prop]
strat_samples.append(sample_prop)
errors_U.append(strat_stderror(U, sample_prop))


In [21]:
strat_df = pd.DataFrame(strat_samples).T
strat_df.columns  = strat_title
strat_df

,equal,100,87,75,prop,equal,120,100,87,75,prop
0,116.317555,100.000000,87.000000,75.000000,126.843712,116.317555,120.000000,100.000000,87.000000,75.000000,126.843712
1,116.317555,100.000000,87.000000,75.000000,13.087877,116.317555,120.000000,100.000000,87.000000,75.000000,13.087877
2,116.317555,149.989529,164.690492,178.260612,183.953510,116.317555,127.372663,149.989529,164.690492,178.260612,183.953510
3,116.317555,115.280692,126.579729,137.009609,141.385123,116.317555,97.897559,115.280692,126.579729,137.009609,141.385123


In [23]:
errors_df = pd.DataFrame(errors_U).T
errors_df.columns = strat_title
errors_df

,equal,100,87,75,prop,equal,120,100,87,75,prop
0,0.042674,0.046057,0.049415,0.053271,0.040850,0.042674,0.042008,0.046057,0.049415,0.053271,0.040850
1,0.045620,0.049237,0.052827,0.056949,0.140906,0.045620,0.044909,0.049237,0.052827,0.056949,0.140906
2,0.027937,0.024578,0.023448,0.022533,0.022179,0.027937,0.026687,0.024578,0.023448,0.022533,0.022179
3,0.020295,0.020387,0.019449,0.018688,0.018394,0.020295,0.022141,0.020387,0.019449,0.018688,0.018394


In [24]:
196*errors_df

,equal,100,87,75,prop,equal,120,100,87,75,prop
0,8.364077,9.027097,9.685376,10.441187,8.006634,8.364077,8.233647,9.027097,9.685376,10.441187,8.006634
1,8.941574,9.650373,10.354103,11.162098,27.617648,8.941574,8.802139,9.650373,10.354103,11.162098,27.617648
2,5.475573,4.817253,4.595847,4.416428,4.347173,5.475573,5.230591,4.817253,4.595847,4.416428,4.347173
3,3.977912,3.995917,3.811914,3.662841,3.605308,3.977912,4.339568,3.995917,3.811914,3.662841,3.605308


In [25]:
100 + 100 + 150 + 115
# GOING FOR THE 100 PER MINORITY CLASS

465